<a href="https://colab.research.google.com/github/vasudevgupta7/gsoc-wav2vec2/blob/main/notebooks/wav2vec2-inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wav2Vec2 inference

In this notebook, we will be using fine-tuned Wav2Vec2 model (with suitable heads) for Automatic Speech Recognition. We will be using `wav2vec2` package from [this repositary](https://github.com/vasudevgupta7/gsoc-wav2vec2).

In [1]:
%%capture
!pip3 install git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main

First download sample `.wav` file & initialize `FILE_PATH`. Additionally, You can directly upload you `.wav` file (in this colab session) for inference on your sample.

In [2]:
!wget https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/Sample2.flac

FILE_PATH = "Sample2.flac"

--2021-06-23 14:50:24--  https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/SA2.wav
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vasudevgupta7/gsoc-wav2vec2/main/data/SA2.wav [following]
--2021-06-23 14:50:24--  https://raw.githubusercontent.com/vasudevgupta7/gsoc-wav2vec2/main/data/SA2.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94252 (92K) [audio/wav]
Saving to: ‘SA2.wav’

SA2.wav             100%[===================>]  92.04K  --.-KB/s    in 0.01s   

2021-06-23 14:50:25 (9.34 MB/s) - ‘SA2.wav’ saved [94252/94252]



Now, you need not worry remaining cells. Just checkout output of the last cell.

## Setup

In [1]:
import tensorflow as tf
from wav2vec2 import Wav2Vec2Processor, Wav2Vec2ForCTC

In [2]:
processor = Wav2Vec2Processor(is_tokenizer=False)
tokenizer = Wav2Vec2Processor(is_tokenizer=True)
model = Wav2Vec2ForCTC.from_pretrained("vasudevgupta/tf-wav2vec2-base-960h")

Loading weights locally from `../vasudevgupta/tf-wav2vec2-base-960h`
Total number of loaded variables: 213


In [25]:
import soundfile as sf

def read_sound(file_name: str):
    if file_name.endswith(".flac"):
      with open(file_name, "rb") as f:
          speech, sample_rate = sf.read(f)
    elif file_name.endswith(".wav"):
      speech, sample_rate = sf.read(file_name)
    else:
      raise NotImplementedError

    assert sample_rate == 16000, "Sound must have a sample rate of 16K"
    speech = tf.constant(speech, dtype=tf.float32)
    return tf.transpose(speech)

def forward(speech: tf.Tensor):
  speech = tf.expand_dims(processor(speech), axis=0)
  tf_out = model(speech, training=False)
  return tf.squeeze(tf.argmax(tf_out, axis=-1))

def recognize_text(speech: tf.Tensor):
  tf_out = forward(speech)
  return tokenizer.decode(tf_out.numpy().tolist())

## Results

In [28]:
from IPython.display import Audio

print(recognize_text(read_sound(FILE_PATH)))
Audio(filename=FILE_PATH)

DON'T ASK ME TO CARRY AN OILY RAG LIKE THAT
